### Rydberg Hamiltonian to MPO

In this notebook, we will demostrate how to construct a Matrix Product Operator (MPO) representation of a Hamiltonian with two-body [Rydberg-Rydberg interaction](https://pulser.readthedocs.io/en/stable/conventions.html#hamiltonians) ,

$$H= \sum _{i} \biggr(\frac{\Omega_i}{2} \sigma_x - \delta _i n_i \biggr) + \sum_ {i<j} c_ {i,j} n_i n_j, $$


This document is intended for developers and curious users who want to understand how to practically implement such a Hamiltonian in a Matrix Product State (MPS) emulator, like Pasqal's emu-mps.

We will introduce the concept of operator-valued matrices, which are key to constructing the MPO. Through practical examples, we aim to clarify the procedure.

This tutorial is organized as follows: The first section, *Rydberg Hamiltonian to MPO*, will cover the basic theory behind our implementation. The next section, *Code Implementation*, will contain the functions that generate the MPO of the Rydberg Hamiltonian. Finally, the *Examples* section will demonstrate the results for 3 and 5 atoms. 

In [1]:
# import libraries
!pip install sympy
import sympy as sp
from sympy.physics.quantum import TensorProduct as TP
from IPython.display import display, Latex

We will use Sympy for the symbolic representation of the elements in the MPO. This will help us understand which operators are applied to each site and where to place the constants that represent the interaction terms.

In the following section, we will define functions that will assist in constructing the MPO.

In [2]:
# creation of the identity and number operators
def iden_op(i:int):
    """Single qubit identity operator"""
    return sp.Symbol(f"𝟙_{i}", commutative=False)

def n_op(i:int):
    """Single qubit number operator"""
    return sp.Symbol(f"n_{i}", commutative=False)

In [3]:
#utility functions
def gate_at(d: dict[int, sp.Symbol], n: int):
    """Utility function for filling operators with identities."""
    return TP(*(
        iden_op(i) if i not in d
        else d[i]
        for i in range(n)
    ))


def mpo_factors_product_pairwise(a: sp.Matrix, b: sp.Matrix):
    """Matrix product where element-wise multiplication is the tensor product."""

    assert sp.shape(a)[1] == sp.shape(b)[0], "Incompatible matrix dimensions"

    common_dim = sp.shape(a)[1]

    res_rows = sp.shape(a)[0]
    res_cols = sp.shape(b)[1]

    res = sp.Matrix([
            [sum(TP(a[row, k], b[k, col]).expand(tensorproduct=True) for k in range(common_dim))
                for col in range(res_cols)
            ]
        for row in range(res_rows)
    ])

    if res_rows == res_cols == 1:
        return res[0, 0]

    return res


def mpo_factors_product(*args):
    """n-ary matrix product where element-wise multiplication is the tensor product."""

    assert len(args) >= 2
    if len(args) == 2:
        return mpo_factors_product_pairwise(*args)
    
    return mpo_factors_product_pairwise(args[0], mpo_factors_product(*args[1:]))

$\newcommand\unity{1\!\!1}$
### Rydberg Hamiltonian to MPO


We aim to represent the Rydberg Hamiltonian, given by:

$$H= \sum _{i} \biggr(\frac{\Omega_i}{2} \sigma_x - \delta _i n_i \biggr) + \sum_ {i<j} c_ {i,j} n_i n_j, $$

where $n$ is the number operator and $\sigma_x$ is the x pauli operator as a Matrix Product Operator or MPO

<div >
<img src ="images/mpofull.png" width = "450" height="100" style="display=block; margin:auto"/>
</div>

As can be seen, we have set the phase to zero ($\phi=0$), which cancels out the $\sigma_y$ term in the original Hamiltonian. However, this will not affect the resulting MPO because, as we will see later, all single-gate terms will be located in a specific position within the MPO.

To facilitate the calculations, let us express a rank-4 tensor

<div >
<img src ="images/tensor1.png" width = "100" height="100" style="display=block; margin:auto"/>
</div> 

as operator-valued matrices, given by  $\begin{bmatrix} 
B_{0,0} & B_{0,1} & \ldots \\ 
B_{1,0} & B_{1,1} & \ldots \\ 
\ldots & \ldots &\ddots
\end{bmatrix}$

where each $B_{i,j}$ has indices $k,l$. Additionally, we define the multiplication of the inner matrices as the Kronecker product $\otimes$. Then, an MPO consists of a series of matrices, and performing matrix multiplication is equivalent to contracting the bonds and reshaping all physical input and output indices into a single “fat” index for input and output, respectively.


#### Single terms

The sum of single-qubit terms such as:
$$
A_i = \left( \frac{\Omega_i}{2} \sigma_i^x - \delta_i n_i \right),
$$

we can implement them with a bond dimension of 2. 

For each qubit $i$, the Hamiltonian term $A_i$​ is represented as a $2\times 2$ matrix in the MPO at position $i$. Thus, the MPO takes the following structure:
\begin{bmatrix}\unity_i & 0 \\ A_i & \unity _i \end{bmatrix}
The overall Hamiltonian MPO for multiple qubits is constructed by taking the matrix product of these individual MPOs



For example, let’s create the MPOs for the Rydberg Hamiltonian of 3 atoms, where only single operators $A_i = X_i$ are applied to each atom:

$$H = \begin{bmatrix} X_1 & \unity_1 \\ \end{bmatrix} \begin{bmatrix}\unity_2 & 0 \\ X_2 & \unity  \end{bmatrix}  \begin{bmatrix}  \unity_3 \\ X_3 \end{bmatrix} $$

$$H = \begin{bmatrix} X_1 \otimes  \unity_2 + \unity_1  \otimes X_1 & \unity_2  \end{bmatrix}  \begin{bmatrix}  \unity_3 \\ X_3 \end{bmatrix}$$

$$H = X_1 \otimes \unity_2 \otimes \unity_3 + \unity_1 \otimes X_2 \otimes \unity_3 +\unity_1 \otimes \unity_2 \otimes X_3 $$

Thus, we can generalize any given 2x2 matrix $A_i$ for $N$ atoms as follows:

$$H = \begin{bmatrix} A_1 & \unity \\ \end{bmatrix} \begin{bmatrix}\unity & 0 \\ A_1 & \unity  \end{bmatrix} \ldots  \begin{bmatrix}\unity & 0 \\ A_{N-1} & \unity  \end{bmatrix} \begin{bmatrix}  \unity \\ A_N \end{bmatrix}
$$ 

Note that $0$ is the zero 2x2 matrix, and $\unity_i$ is the 2x2 identity matrix.

In summary, the single operators will always appear in these specific positions in the MPO, regardless of the operator(s) applied.

### 

#### Rydberg Hamiltonian

There are various ways to obtain the MPO for the Rydberg Hamiltonian ([lecture notes, L08](https://www2.physik.uni-muenchen.de/lehre/vorlesungen/sose_23/tensor_networks_23/skript/index.html) or check [chapter 3, ](https://arxiv.org/pdf/1008.3477)) . However, the most efficient method so far, and the one we use in EMU-MPS, is as follows. The MPO of the Rydberg Hamiltonian can be expressed as a combination of matrices $S$, $L$, $M$, $R$, and $E$:

$$H = S L_1 \ldots L_a M R_b \ldots R_1 E, $$ 

where $a+b+3=N$, with $a=\lfloor \frac{N-2}{2} \rfloor$, and $N$ is the number of atoms. The matrices are defined as follows:

 - $S = \begin{bmatrix} A_1 & \unity_1  & n_1  \end{bmatrix}$ is the first MPO matrix

 - $E = \begin{bmatrix}  \unity_N   \\ A_N  \\ n_N  \end{bmatrix}$ unless $N=2$,  $E = \begin{bmatrix}  \unity_2   \\ A_2  \\ C_{12}n_2  \end{bmatrix}$,  with $\unity_i$ being the 2x2 identity matrix at atom $i$, $n_i$ is the number operator at atom $i$ and $E$ is always located at the end.
 

The left-term matrices $L_i$ are defined as:

$L_i = \ \ \overset{\text{3+i} \longrightarrow}{  \stackrel{2+i \downarrow}{}{ \begin{bmatrix} \unity_{i+1} & \mathbb{0} & 0 & \mathbb{0} & 0 & 0 \\ A_{i+1} & \unity_{i+1} &\mathbb{0} &\dots& 0 &n_{i+1} \\
C_{1,i+1} n_{i+1} & 0 & \unity_{i+1} & 0 & \ldots &0 \\
 \vdots & \vdots & \vdots & \ddots &\ldots & 0 \\
 C_{k,i+1} n_{i+1} & 0& 0 &\ldots & \unity_{i+1} &0
\end{bmatrix}}}
$ with $k= 1,\ldots, i$, and $C_{i,j}$ indicates the interaction between atom $i$ and $j$. $L_i$ matrices grow from left to right (until matrix $M$). 


The right-term matrices $R_i$ are given by:

$ R_i = \ \ \ \ \overset{\text{2+i} \longrightarrow}{ \stackrel{3+i \downarrow}{}{ \begin{bmatrix} \unity_k & 0 &0&\ldots &  0 \\ 
A_k & \unity_k & C_{k,k+1} n_k &\ldots &C_{k,k+b}n_k \\
n_k & 0 & 0 & \dots&0 \\
0 & 0 &\unity_k &  \ldots&0\\
\vdots & \vdots & \vdots & \ddots & 0 \\
0 & 0 & \dots & \ldots & \unity_k
\end{bmatrix} }}
$
with $k=a+b+i+1$. The $R_i$ matrix grows from right to left (until matrix $M$)


The middle matrix $M$ is defined as:

$M = \begin{bmatrix}\unity_{a+2}  & 0 & 0 \ldots & 0 &0 \\
A_{a+2} & \unity_{a+2} & C_{a+2,a+3}n_{a+2} & \ldots & C_{a+2,N}n_{a+2}\\
C_{1,a+2}n_{a+2} & 0 & C_{i,j} \unity_{a+2} & C_{i,j+1} \unity_{a+2} &\ldots\\
\vdots & 0 & C_{i+1,j} \unity_{a+2} & C_{i+1,j+1} \unity_{a+2} &\ldots\\
C_{a+1,a+2} n_{a+2} & 0 &\vdots & \ldots &\ddots
\end{bmatrix}$ where the Bloch of $C_{i,j} \unity$ with $i<a+2$ and $j>a+2$ are the interaction terms for past and future interactions. 

These matrices are written in EMU-MPS, however, for this tutorial we are going to use a simplified version of them. 

#### Code Implementation

In the following section, the examples will ilustrate how all these matrices forms the MPO. 

In [4]:
#implementation of S, E, Li, Rj and E
def _first_factor_rydberg(gate: sp.Symbol)->sp.Matrix:
    """
    Creates the first Rydberg Hamiltonian factor.
    """
    fac = sp.zeros(1, 3) 
    fac[0, 1] = iden_op(0)
    fac[0, 2] = n_op(0)  # number operator

    fac[0, 0] = gate
    return fac


def _last_factor_rydberg(gate: sp.Symbol, scale: float | complex, num_atoms: int)->sp.Matrix:
    """
    Creates the last Rydberg Hamiltonian factor.
    """
    fac = sp.zeros(3, 1)
    fac[0, 0] = iden_op(num_atoms)
    fac[2, 0] = scale * n_op(num_atoms)

    fac[1, 0] = gate
    return fac


def _left_factor_rydberg(gate: sp.Symbol, scales: list[sp.Symbol], num_atom: int)->sp.Matrix:
    """
    Creates the Rydberg Hamiltonian factors in the left half of the MPS, excepted the first factor.
    """
    index = len(scales)
    fac = sp.zeros(
        index + 2,
        index + 3,
    )
    for i, val in enumerate(scales):
        fac[i + 2, 0] = val * n_op(num_atom)  # interaction with previous qubits
    fac[1, index + 2] = n_op(num_atom)  #  interaction with next qubits
    for i in range(index + 2):
        fac[i, i] = iden_op(
            num_atom
        )  # identity matrix to carry the gates of other qubits

    fac[1, 0] = gate
    return fac


def _right_factor_rydberg(gate: sp.Symbol, scales: list[sp.Symbol], num_atom: int)->sp.Matrix:
    """
    Creates the Rydberg Hamiltonian factors in the right half of the MPS, excepted the last factor.
    """
    index = len(scales)
    fac = sp.zeros(index + 3, index + 2)
    for i, val in enumerate(scales):
        fac[1, i + 2] = val * n_op(num_atom)  # XY interaction with previous qubits
    fac[2, 0] = n_op(num_atom)  # XY interaction with next qubits
    for i in range(2, index + 2):
        fac[i + 1, i] = iden_op(num_atom)
    fac[0, 0] = iden_op(
        num_atom
    )  # identity to carry the next gates to the previous qubits
    fac[1, 1] = iden_op(num_atom)  # identity to carry previous gates to next qubits

    fac[1, 0] = gate
    return fac


def _middle_factor_rydberg(
    gate: sp.Symbol,
    scales_l: list[sp.Symbol],
    scales_r: list[sp.Symbol],
    scales_mat: list[list[sp.Symbol]],
    num_atom:int
)->sp.Matrix:
    """
    Creates the Rydberg Hamiltonian factor at index ⌊n/2⌋ of the n-qubit MPO.
    """
    assert len(scales_mat) == len(scales_l)
    assert all(len(x) == len(scales_r) for x in scales_mat)

    fac = sp.zeros(
        len(scales_l) + 2,
        len(scales_r) + 2,
    )
    for i, val in enumerate(scales_r):
        fac[1, i + 2] = val * n_op(num_atom)  # rydberg interaction with previous qubits
    for i, val in enumerate(scales_l):
        fac[i + 2, 0] = val * n_op(num_atom)  # rydberg interaction with next qubits
    for i, row in enumerate(scales_mat):
        for j, val in enumerate(row):
            fac[i + 2, j + 2] = (
                val * iden_op(num_atom)
            )  # rydberg interaction of previous with next qubits
    fac[0, 0] = iden_op(num_atom)  # identity to carry the next gates to the previous qubits
    fac[1, 1] = iden_op(num_atom)  # identity to carry previous gates to next qubits

    fac[1, 0] = gate
    return fac

In [5]:
def general_make_H(interaction_matrix:list[list[sp.Symbol]],single_qubit_term:list[sp.Symbol]):
    """Based on make_H function of emu-mps. 
    Constains the basics functions that creates the MPO for the 
    Hamiltonian: H = SL1...La M R1 .... Rb E """

    
    nqubits = interaction_matrix.shape[0]
    cores = [_first_factor_rydberg(single_qubit_term[0])]
    if nqubits > 2:
        for i in range(1, nqubits // 2):

            cores.append(
                _left_factor_rydberg(
                    single_qubit_term[i],
                    [interaction_matrix[j, i] for j in range(i)],
                i)
            )

        i = nqubits // 2
        cores.append(
            _middle_factor_rydberg(
                single_qubit_term[i],
                [interaction_matrix[j, i] for j in range(i)],
                [interaction_matrix[i, j] for j in range(i + 1, nqubits)],
                [
                    [interaction_matrix[k, j] for j in range(i + 1, nqubits)]
                    for k in range(i)
                ],
            i)
        )

        for i in range(nqubits // 2 + 1, nqubits - 1):
            cores.append(
                _right_factor_rydberg(
                    single_qubit_term[i],
                    [interaction_matrix[i, j] for j in range(i + 1, nqubits)],
                i)
            )

    scale = 1 # int for printing with sympy 
    if nqubits == 2:
        scale = interaction_matrix[0, 1]
    cores.append(
        _last_factor_rydberg(
            single_qubit_term[-1],
            scale,nqubits-1
        )
    )
    return cores


In [6]:
# declaring a symbol A for single site operator and U for the interaction coefficient
def A(i: int)->sp.Symbol:
    """Single qubit terms"""
    return sp.Symbol(f"A_{i}", commutative=False)

def U(i: int, j: int)->sp.Symbol:
    """Interaction coefficient for i,j atoms"""
    return sp.Symbol(f"U_{i}{j}")

In [7]:
#interaction matrix 
def general_interaction_matrix(num_atoms:int)->sp.Matrix:
    """" For this tutorial purpuses: generates a symmetric matrix where its elements Uᵢⱼ represents the Rydberg interaction
    between atom i and atom j."""
    interaction_matrix = sp.zeros(num_atoms,num_atoms)
    for numi in range(num_atoms):
        for numj in range(numi + 1, num_atoms):
            interaction_matrix[numi,numj] = U(numi,numj)
            interaction_matrix[numj, numi] = interaction_matrix[numi, numj] # for symmetry 
    return interaction_matrix

### Examples

##### Example using 3 atoms
The MPO for 3 atoms should contain 3 matrices: $S, M,E$. We should feed the `general_make_H` function with 3 single operators terms like [A(0),A(1),A(2)] that act on each atom and the respective interaction matrix `inter_matri`. 

In [8]:
single_terms = [A(0),A(1),A(2)]
inter_matri = general_interaction_matrix(3)
cores = general_make_H(inter_matri,single_terms)
print("Matrix S:")
display(cores[0])
print("Matrix M:")
display(cores[1])
print("Matrix E:")
display(cores[2])

Matrix S:


Matrix([[A_0, 𝟙_0, n_0]])

Matrix M:


Matrix([
[     𝟙_1,   0,        0],
[     A_1, 𝟙_1, U_12*n_1],
[U_01*n_1,   0, U_02*𝟙_1]])

Matrix E:


Matrix([
[𝟙_2],
[A_2],
[n_2]])

The final Rydberg Hamiltonian with all its terms: 

In [9]:
mpo_factors_product(*cores)

U_01*n_0xn_1x𝟙_2 + U_02*n_0x𝟙_1xn_2 + U_12*𝟙_0xn_1xn_2 + A_0x𝟙_1x𝟙_2 + 𝟙_0xA_1x𝟙_2 + 𝟙_0x𝟙_1xA_2

##### For testing purposes
We are going to create a Rydberg Hamiltonian and test it with our MPO implementation


In [10]:
def reference_hamiltonian(qubit_count: int):
    """For testing puruposes: creates the Rydberg Hamiltonian """
    result = sum(gate_at({i: A(i)}, qubit_count) for i in range(qubit_count))

    result += sum(
        U(j, i) *
        gate_at({
            i: n_op(i),
            j: n_op(j)
        }, qubit_count)
        for i in range(qubit_count) for j in range(i)
    )

    return result

In [11]:
## simple testing the MPO for 3 atoms with the reference Hamiltonian
str(reference_hamiltonian(3)) == str(mpo_factors_product(*cores))

True

##### Example for 5 atoms
The MPO for 5 should contain 5 matrices: $S, L_1, M, R_1,E$

In [12]:
single_terms = [A(0),A(1),A(2),A(3),A(4)]
inter_matri = general_interaction_matrix(5)
cores = general_make_H(inter_matri,single_terms)
print("Matrix S:")
display(cores[0])
display(Latex(f"Matrix $L_1$:"))

display(cores[1])
print("Matrix M:")
display(cores[2])
display(Latex(f"Matrix $R_1$:"))
display(cores[3])
print("Matrix E:")
display(cores[4])

Matrix S:


Matrix([[A_0, 𝟙_0, n_0]])

<IPython.core.display.Latex object>

Matrix([
[     𝟙_1,   0,   0,   0],
[     A_1, 𝟙_1,   0, n_1],
[U_01*n_1,   0, 𝟙_1,   0]])

Matrix M:


Matrix([
[     𝟙_2,   0,        0,        0],
[     A_2, 𝟙_2, U_23*n_2, U_24*n_2],
[U_02*n_2,   0, U_03*𝟙_2, U_04*𝟙_2],
[U_12*n_2,   0, U_13*𝟙_2, U_14*𝟙_2]])

<IPython.core.display.Latex object>

Matrix([
[𝟙_3,   0,        0],
[A_3, 𝟙_3, U_34*n_3],
[n_3,   0,        0],
[  0,   0,      𝟙_3]])

Matrix E:


Matrix([
[𝟙_4],
[A_4],
[n_4]])

The final Rydberg Hamiltonian with all its terms is: 

In [13]:
mpo_factors_product(*cores)

U_01*n_0xn_1x𝟙_2x𝟙_3x𝟙_4 + U_02*n_0x𝟙_1xn_2x𝟙_3x𝟙_4 + U_03*n_0x𝟙_1x𝟙_2xn_3x𝟙_4 + U_04*n_0x𝟙_1x𝟙_2x𝟙_3xn_4 + U_12*𝟙_0xn_1xn_2x𝟙_3x𝟙_4 + U_13*𝟙_0xn_1x𝟙_2xn_3x𝟙_4 + U_14*𝟙_0xn_1x𝟙_2x𝟙_3xn_4 + U_23*𝟙_0x𝟙_1xn_2xn_3x𝟙_4 + U_24*𝟙_0x𝟙_1xn_2x𝟙_3xn_4 + U_34*𝟙_0x𝟙_1x𝟙_2xn_3xn_4 + A_0x𝟙_1x𝟙_2x𝟙_3x𝟙_4 + 𝟙_0xA_1x𝟙_2x𝟙_3x𝟙_4 + 𝟙_0x𝟙_1xA_2x𝟙_3x𝟙_4 + 𝟙_0x𝟙_1x𝟙_2xA_3x𝟙_4 + 𝟙_0x𝟙_1x𝟙_2x𝟙_3xA_4

In [14]:
#simple testing the Hamiltonian generated
str(reference_hamiltonian(5)) == str(mpo_factors_product(*cores))

True